In [1]:
# Scraping
import requests
from bs4 import BeautifulSoup

# Data manipulation
import pandas as pd

In [ ]:
base_url = 'http://soccer.phew.homeip.net/team/stats/yearly/chart/?start=0&sort=position&year=2023'

# List of teams that belongs to clubs in the J-league (J1, J2 & J3)
jteams = ['%BB%A5%CB%DA', '%C8%AC%B8%CD', '%B4%E4%BC%EA', '%C0%E7%C2%E6', '%BD%A9%C5%C4', '%BB%B3%B7%C1', '%CA%A1%C5%E7', '%A4%A4%A4%EF%A4%AD', '%BC%AF%C5%E7', '%BF%E5%B8%CD', '%C6%CA%CC%DA', '%B7%B2%C7%CF', '%B1%BA%CF%C2', '%C2%E7%B5%DC', '%C0%E9%CD%D5', '%C7%F0', 'FC%C5%EC%B5%FE', '%C5%EC%B5%FEV', '%C4%AE%C5%C4', '%C0%EE%BA%EA', '%B2%A3%C9%CDFM', '%B2%A3%C9%CDFC', 'YS%B2%A3%C9%CD', '%BE%C5%C6%EE', '%C1%EA%CC%CF%B8%B6', '%B9%C3%C9%DC', '%BE%BE%CB%DC', '%C4%B9%CC%EE','%BF%B7%B3%E3','%C9%D9%BB%B3','%B6%E2%C2%F4','%C0%B6%BF%E5','%C8%D8%C5%C4','%C6%A3%BB%DE','%BE%C2%C4%C5','%CC%BE%B8%C5%B2%B0','%B4%F4%C9%EC','%B5%FE%C5%D4','G%C2%E7%BA%E5','C%C2%E7%BA%E5','FC%C2%E7%BA%E5&','%C6%E0%CE%C9','%BF%C0%B8%CD','%C4%BB%BC%E8','%B2%AC%BB%B3','%B9%AD%C5%E7','%BB%B3%B8%FD','%BB%BE%B4%F4','%C6%C1%C5%E7','%B0%A6%C9%B2','%BA%A3%BC%A3','%CA%A1%B2%AC','%CB%CC%B6%E5%BD%A3','%C4%BB%C0%B4','%C4%B9%BA%EA','%B7%A7%CB%DC','%C2%E7%CA%AC','%B5%DC%BA%EA','%BC%AF%BB%F9%C5%E7','%CE%B0%B5%E5']

data_list = [] 

In [1]:
for team in jteams:
    url = f"{base_url}&team={team}"
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html5lib")

    table = soup.find("table", {"id": "main_table"})

    club_name = soup.find("div", {"class": "club_info"}).b.text.strip()

    for row in table.find_all("tr")[1:]:

        cells = row.find_all("td", recursive=False)

        if len(cells) < 5:
            continue

        data = {
            'クラブ': club_name,  # Club
            '背番号': cells[0].text,  # Kit Number
            'ポジション': cells[1].text,  # Player Position
            '名前': cells[2].text.replace("\u3000", " "),  # Name
            '最終出場': cells[3].text, # Last Appearance
            'フル出場': 0,  # Time Played in Full Match 
            '控え': 0,  # Time Spent as Reserve
            '途中出場': 0,  # Time since Subbed On
            '途中退場': 0,  # Time since Subbed off 
            'メンバー外': 0  # Time as Non-squad Member
        }

        last_cell = cells[-1]
        nested_table = last_cell.find_all("td")

        for td in nested_table:
            td_class = td.get("class")[0]
            td_value = int(td.get("width"))
            if td_class == "full":
                data["フル出場"] = td_value
            elif td_class == "sub":
                data["控え"] = td_value
            elif td_class == "in":
                data["途中出場"] = td_value
            elif td_class == "out":
                data["途中退場"] = td_value
            elif td_class == "other":
                data["メンバー外"] = td_value

        data_list.append(data) 

df = pd.DataFrame(data_list)

In [2]:
df

,クラブ,背番号,ポジション,名前,最終出場,フル出場,控え,途中出場,途中退場,メンバー外
0,北海道コンサドーレ札幌,1,GK,菅野 孝憲,04/29,270,0,0,0,2790
1,北海道コンサドーレ札幌,21,GK,松原 修平,-,0,90,0,0,2970
2,北海道コンサドーレ札幌,25,GK,ク ソンユン,04/15,630,90,0,0,2340
3,北海道コンサドーレ札幌,48,GK,大谷 幸輝,-,0,720,0,0,2340
4,北海道コンサドーレ札幌,2,DF,田中 駿汰,04/29,900,0,0,0,2160
...,...,...,...,...,...,...,...,...,...,...
1938,ＦＣ琉球,9,FW,野田 隆之介,04/29,0,0,107,195,3118
1939,ＦＣ琉球,16,FW,阿部 拓馬,04/15,0,0,113,63,3244
1940,ＦＣ琉球,34,FW,ケルヴィン,04/29,0,0,45,145,3230
1941,ＦＣ琉球,35,FW,サダム スレイ,-,0,0,0,0,3420


In [35]:
#df.to_excel('playing-time-stats.xlsx', index=False)